In [ ]:
from pyspark.sql import SparkSession
from delta import *

builder = SparkSession.builder.appName("Spark w/ Delta") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
df = spark.read.load('./products.csv', format='csv', header=True)

delta_table_path = "./delta/products-delta"
df.write.format("delta").save(delta_table_path)

In [ ]:
 from delta.tables import *
 from pyspark.sql.functions import *

 # Create a deltaTable object
 deltaTable = DeltaTable.forPath(spark, delta_table_path)

 # Update the table (reduce price of product 771 by 10%)
 deltaTable.update(
     condition = "ProductID == 771",
     set = { "ListPrice": "ListPrice * 0.9" })

 # View the updated data as a dataframe
 deltaTable.toDF().show(10)

In [ ]:
 new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
 new_df.show(10)

In [ ]:
 deltaTable.history(10).show(20, False, True)